# Configurar algoritmo COF

En este ejemplo vamos a probar diferentes parámetros del algoritmo COF para resolver un problema concreto. Muchas veces es necesario hacer un ajuste para cada problema debido a que presentan características muy diferentes.

Para analizar el algoritmo vamos a evaluar las medidas: sensibilidad y especificación, para conseguir un compromiso entre ambas. Para su análisis se va a realizar una división del conjunto de datos, en datos de train y datos de test (como se han visto en todos los ejemplos estudiados) con la proporción (70% para training y 30% para test). Al ser un método estocástico, se deben realizar diferentes ejecuciones y trabajar con los valores medios, no obstante, en este pequeño estudio no llevaremos a cabo diferentes ejecuciones

El algoritmo que se va a configurar es:
        COF: Connection Outlier Factor




Paso 1. Instalar la librería PyOD

In [ ]:
!pip install PyOD

Paso 2. Importamos las librerías que necesitamos

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.cof import COF
from pyod.utils.utility import standardizer
from sklearn.metrics import confusion_matrix

import os
import sys

Paso 3. Definimos la lista de datos que vamos a usar (3 en nuestro caso), previamente debemos haberlos cargado en google colaboration (como se ha explicado en el video), la semilla del generador de números aleatorios que se usará en los casos que sea necesario y las variables donde almacenaremos las métricas que se van a obtener para cada ejecución por cada algoritmo, se incluye el tiempo de cómputo.

In [ ]:
# Definimos la lista de datos que vamos a usar 
mat_datasets = ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat']


# Definimos las variables que se van a utilizar para almacenar los resultados de 
# cada conjunto de datos, todas tendrán las mismas columnas y se hará una
# tabla para comparar cada métrica de las estudiadas
columnas_tabla = ['Algoritmo','Datos', '#Ejemplos', '#Dimensiones', 'Anomalías(%)','k',
              'Se','Sp']



Paso 4. Vamos a ejecutar el algoritmos con cada conjunto de datos y con diferente valor de k, primero preparamos los datos normalizándolos y dividiéndolos en train y test según la proporción (70% y 30%), a continuación aplicamos el algoritmo variando el parámetro que queremos estudiar.



In [ ]:
# Cargamos el conjunto de datos que vamos a evaluar
mat_dataset = mat_datasets[0]
result_tabla = pd.DataFrame(columns=columnas_tabla)
dataset = loadmat(os.path.join('sample_data', mat_dataset))

# Definimos la semilla que se va a utilizar, siendo un número aleatorio entre 1 y 50
random_state = np.random.RandomState(50)


# Separamos los atributos de la clase
X = dataset['X']
y = dataset['y'].ravel()

# Obtenemos la fracción de anomalías que tiene el conjunto de datos
# es una variable que necesitan los algoritmos
anomalias_fraccion = np.count_nonzero(y) / len(y)
anomalias_porcentaje = round(anomalias_fraccion * 100, ndigits=4)

# Divimos los datos en train y test (70%, 30% respectivamente). También
# se pueden usar otras formas de cross-validation 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                        random_state=random_state)

# Normalizamos los datos (usamos el métodos de PyOD)
X_train_norm, X_test_norm = standardizer(X_train, X_test)

   
# Para cada configuración definido
for k in range(10, 100, 10):
      # Configuramos los algoritmos que vamos a usar
       clf_name = 'COF'
       clf = COF(contamination=anomalias_fraccion, n_neighbors = k)
       
       clf.fit(X_train_norm)
       
       # Obtenemos los resultados para los datos de train y test
       y_train_pred = clf.labels_ 
       y_test_pred = clf.predict(X_test_norm)  
       
       #Calculamos sensibilidad y especificidad  desde la matriz de confusión
       # para los datos de  test
       cm = confusion_matrix(y_test,y_test_pred)
       total=sum(sum(cm))
       sensitivity = cm[1,1]/(cm[1,0]+cm[1,1])
       specificity = cm[0,0]/(cm[0,0]+cm[0,1])
       
       se = round(sensitivity, ndigits=4)
       sp = round(specificity, ndigits=4)
       
       #print('{clf_name}, k:{k} Se:{se}, Sp:{sp}'.format(clf_name=clf_name, k=k, se=se, sp=sp))
       
       # Definimos las listas para almacenar los resultados
       result_list = [clf_name,mat_dataset[:-4], X.shape[0], X.shape[1], anomalias_porcentaje]
      
       result_list.append(k)
       result_list.append(se)
       result_list.append(sp)
       
       # Una vez finalizadas las ejecuciones de todos los métodos,
       # pasamos la información a tabla
       temp_tabla = pd.DataFrame(result_list).transpose()
       temp_tabla.columns = columnas_tabla
       result_tabla = pd.concat([result_tabla, temp_tabla], axis=0)

print('Resultado COF')
result_tabla


Resultado COF


,Algoritmo,Datos,#Ejemplos,#Dimensiones,Anomalías(%),k,Se,Sp
0,COF,glass,214,9,4.2056,2,0.0,0.9032
0,COF,glass,214,9,4.2056,3,0.0,0.9194
0,COF,glass,214,9,4.2056,4,0.3333,0.9032
0,COF,glass,214,9,4.2056,5,0.3333,0.9194
0,COF,glass,214,9,4.2056,6,0.0,0.9677
0,COF,glass,214,9,4.2056,7,0.0,0.9839
0,COF,glass,214,9,4.2056,8,0.0,0.9839
0,COF,glass,214,9,4.2056,9,0.0,0.9839
0,COF,glass,214,9,4.2056,10,0.0,0.9839
0,COF,glass,214,9,4.2056,11,0.0,0.9839
